In [1]:

import os
import random
from time import time
import pandas as pd
import numpy as np
import networkx as nx
import graph_tool.all as gt

def build_net(workdir, res):

    datadir = f"{workdir}/data_{res}"
    edge_df = pd.read_csv(f"{datadir}/edge.txt", delimiter="\t")
    node_df = pd.read_csv(f"{datadir}/node.txt", delimiter="\t")
    edge_df = edge_df.astype({"id1":int, "id2":int})
    node_df = node_df.astype({"id":int})
    edge_df["sig"]=-np.log10(edge_df["pvalue"].to_numpy())

    #print(edge_df.head())
    #g.add_nodes_from(node_df["id"].to_numpy())
    g = nx.from_pandas_edgelist(edge_df, "id1", "id2", ["counts", "isPeak1", "isPeak2", "sig"])
    for i, row in node_df.iterrows():
        nx.set_node_attributes(g, {row['id']: {'pore': row['pore'],
                                               'hmmdomain': row['hmmdomain'],
                                               'chrom': row['chrom'],
                                               'start': row['s'],
                                               'end': row['e']}})
    return g

def build_pronet(regnet):
    # Step 1: Create a copy of the original graph
    pronet = regnet.copy()

    # Step 2: Add edge attribute 'isProLink' based on node 'hmmdomain'
    for u, v in pronet.edges():
        if (pronet.nodes[u]['hmmdomain'] in ["AP", "IP"] and 
            pronet.nodes[v]['hmmdomain'] in ["AP", "IP"]):
            pronet[u][v]['isProLink'] = True
        else:
            pronet[u][v]['isProLink'] = False

    # Step 3: Remove edges where 'isProLink' is False
    edges_to_remove = [edge for edge in pronet.edges() if not pronet.edges[edge]['isProLink']]
    pronet.remove_edges_from(edges_to_remove)

    # Step 4: Count and remove isolated nodes (degree 0)
    isolated_nodes = [node for node in pronet.nodes() if pronet.degree(node) == 0]
    counter = len(isolated_nodes)
    print(f"{counter} nodes have no edges")
    pronet.remove_nodes_from(isolated_nodes)

    # Step 5: Print statistics after edge purge
    print(f"Number of nodes after edge-purge: {pronet.number_of_nodes()}")
    print(f"Number of edges after edge-purge: {pronet.number_of_edges()}")

    # Step 6: Add node attribute 'isPro' based on degree
    for node in pronet.nodes():
        pronet.nodes[node]['isPro'] = pronet.degree(node) > 0

    # Step 7: Remove nodes where 'isPro' is False
    nodes_to_remove = [node for node in pronet.nodes() if not pronet.nodes[node]['isPro']]
    pronet.remove_nodes_from(nodes_to_remove)

    # Step 8: Print final statistics
    # Note: In the original, np.sum(pronet.vp["isPro"].a) prints the count of True values.
    # Here, we can simply use the number of nodes since all remaining nodes have 'isPro' = True.
    print(f"{pronet.number_of_nodes()}")  # Equivalent to np.sum(pronet.vp["isPro"].a)
    print(f"Number of nodes after node-purge: {pronet.number_of_nodes()}")
    print(f"Number of edges after node-purge: {pronet.number_of_edges()}")

    return pronet

def build_pronet2(regnet):
    # Step 1: Create a copy of the original graph
    pronet = regnet.copy()

    print(f"Number of nodes before edge-purge: {pronet.number_of_nodes()}")
    print(f"Number of edges before edge-purge: {pronet.number_of_edges()}")

    nodes_to_remove = [node for node in pronet.nodes() if not (pronet.nodes[node]['hmmdomain'] in ["AP", "IP"])]
    pronet.remove_nodes_from(nodes_to_remove)

    print(f"Number of nodes after node-purge: {pronet.number_of_nodes()}")
    print(f"Number of edges after node-purge: {pronet.number_of_edges()}")

    return pronet

figroot = "/mirror/Data/Gengyao_Chen/TRIP_article_figures/PE_network/network_properties/Figures"
dataroot = "/mirror/Data/Gengyao_Chen/TRIP_article_figures/PE_network/network_properties/Data"


In [2]:
enhnet_workdir = "/mirror/Data/Gengyao_Chen/TRIP_article_figures/PE_network/25-01-17/EnhNet"
#regnet_workdir = "/mirror/Data/Gengyao_Chen/TRIP_article_figures/PE_network/25-01-17/RegNet"
#pronet_workdir = "/mirror/Data/Gengyao_Chen/TRIP_article_figures/PE_network/25-01-17/ProNet"
enhnet = build_net(enhnet_workdir, 500)
#regnet = build_net(regnet_workdir, 250)
#pronet = build_pronet2(regnet)

In [3]:
import networkx as nx
import numpy as np
import pandas as pd
from networkx.algorithms import community
from community import community_louvain

########################################
# 1. Find CCs, cliques and communities
########################################

# Assume enhnet is an undirected NetworkX graph already built
# For example: enhnet = nx.read_gpickle("path_to_enhnet.gpickle")

# Connected components (each as a set of nodes)
cc_list = list(nx.connected_components(enhnet))
# Compute CC sizes and sort (largest first)
cc_sizes = [len(cc) for cc in cc_list]
sorted_idx = np.argsort(cc_sizes)[::-1]
sorted_cc_list = [cc_list[i] for i in sorted_idx]
sorted_sizes = np.array(cc_sizes)[sorted_idx]

# Compute size thresholds based on sorted sizes
# st50: size of 50th largest (index 49)
# st1000: size of 1000th largest (index 999) -- provided that there are at least 1000 CCs.
if len(sorted_sizes) >= 1000:
    st50 = sorted_sizes[49]
    st1000 = sorted_sizes[999]
else:
    st50 = sorted_sizes[min(49, len(sorted_sizes)-1)]
    st1000 = sorted_sizes[min(999, len(sorted_sizes)-1)]

mcid334 = 46
mcid649 = 23
# Classify connected components by size:
# SCID: size >= st50; MCID: size < st50 but >= st1000; WCID: size < st1000.
cc_class = {}
for idx, cc in enumerate(sorted_cc_list):
    size = len(cc)
    if size >= st50:
        cc_class[idx] = "SCID"
    elif size >= mcid334:
        cc_class[idx] = "MCIDla"
    elif size >= mcid649:
        cc_class[idx] = "MCIDmo"
    elif size >= st1000:
        cc_class[idx] = "MCIDsm"
    else:
        cc_class[idx] = "WCID"



In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
from community import community_louvain

# Create mapping from node to (CC_index, CC_type)
node2cc = {}
for idx, cc in enumerate(sorted_cc_list):
    for n in cc:
        node2cc[n] = (idx, cc_class[idx])

########################################
# 2. Global Communities and Global Cliques
########################################

# Compute global communities using Louvain with weight="counts"
global_communities = community_louvain.best_partition(enhnet, weight="counts")

# Compute global cliques (maximal cliques) and assign first-found clique index per node
global_cliques = list(nx.find_cliques(enhnet))
node2globalclique = {}
for cliq_idx, clique in enumerate(global_cliques):
    for n in clique:
        if n not in node2globalclique:
            node2globalclique[n] = [cliq_idx]
        else:
            node2globalclique[n].append(cliq_idx)

########################################
# 3. Local (CC-wise) Communities and Cliques
########################################

# For each connected component (cc), compute local community and local cliques.
node2localcomm = {}
node2localclique = {}

for idx, cc in enumerate(sorted_cc_list):
    subG = enhnet.subgraph(cc).copy()
    # Compute local communities in the cc using Louvain
    local_comm = community_louvain.best_partition(subG, weight="counts")
    for n in subG.nodes():
        node2localcomm[n] = "cc" + str(idx) + "." + str(local_comm[n])
    # Compute local cliques in the cc (they should be same as when computed globally)
    local_cliques = list(nx.find_cliques(subG))
    for cliq_idx, clique in enumerate(local_cliques):
        for n in clique:
            if n not in node2localclique:
                node2localclique[n] = ["cc" + str(idx) + "." + str(cliq_idx)]
            else:
                node2localclique[n].append("cc" + str(idx) + "." + str(cliq_idx))

In [5]:
global_cliques

[[0, 1],
 [1, 10],
 [1, 14],
 [2, 3],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 9],
 [3, 10],
 [3, 11],
 [3, 12],
 [11, 13],
 [15, 64, 62],
 [15, 63],
 [16, 30],
 [17, 19],
 [18, 20],
 [21, 22],
 [23, 38, 62, 27],
 [23, 38, 62, 29],
 [23, 38, 62, 30],
 [24, 26],
 [24, 28, 61],
 [24, 28, 62],
 [24, 38, 61, 32],
 [24, 38, 61, 33],
 [24, 38, 61, 27],
 [24, 38, 61, 29],
 [24, 38, 61, 31],
 [24, 38, 62, 32],
 [24, 38, 62, 33],
 [24, 38, 62, 27],
 [24, 38, 62, 29],
 [24, 38, 62, 30],
 [24, 38, 62, 31],
 [25, 62],
 [29, 38, 60, 61],
 [30, 37],
 [30, 38, 59],
 [30, 38, 60],
 [31, 38, 60, 61],
 [33, 40, 61],
 [33, 37],
 [33, 38, 34],
 [33, 38, 35],
 [33, 38, 60, 61],
 [36, 62],
 [38, 64, 62],
 [38, 45],
 [38, 46],
 [38, 54, 62],
 [38, 54, 55],
 [38, 57],
 [38, 58],
 [39, 42],
 [39, 60],
 [40, 44],
 [41, 43],
 [45, 47],
 [48, 50],
 [49, 51],
 [50, 53],
 [51, 58],
 [52, 54],
 [53, 58],
 [54, 56],
 [56, 60],
 [65, 67],
 [65, 68],
 [66, 67],
 [69, 76],
 [70, 76],
 [71, 76],
 [72, 76],
 [73,

In [6]:


########################################
# 4. Build anno_node_df
########################################

node_data = []
for n, attr in enhnet.nodes(data=True):
    d = {"node": n}
    d.update(attr)
    # Add connected component info if available
    if n in node2cc:
        d["CC_index"] = node2cc[n][0]
        d["CC_type"]  = node2cc[n][1]
    else:
        d["CC_index"] = np.nan
        d["CC_type"] = np.nan
    # Add global community and clique membership
    d["global_comm"] = global_communities.get(n, np.nan)
    d["global_clique"] = node2globalclique.get(n, np.nan)
    # Add local community and clique membership
    d["local_comm"] = node2localcomm.get(n, np.nan)
    d["local_clique"] = node2localclique.get(n, np.nan)
    node_data.append(d)
anno_node_df = pd.DataFrame(node_data)



In [9]:
import os

# Assuming global_cliques and anno_node_df are already defined
# global_cliques is a list of lists containing node_ids
# anno_node_df has columns 'node', 'chrom', 'start', and 'end'

# Create a directory to store the .aca files
output_dir = f"{dataroot}/cliques"

# Split global_cliques by their element list sizes
cliques_by_size = {}
for clique in global_cliques:
    size = len(clique)
    if size not in cliques_by_size:
        cliques_by_size[size] = []
    cliques_by_size[size].append(clique)

# Process each clique size
for clique_size, cliques in cliques_by_size.items():
    # Open the .aca file for writing
    with open(os.path.join(output_dir, f"clique_{clique_size}.aca"), 'w') as f:
        # Process each clique of the current size
        for cur_node_lst in cliques:
            # Query anno_node_df for chrom, start, and end
            nodes_info = anno_node_df[anno_node_df['node'].isin(cur_node_lst)][['chrom', 'start', 'end']]
            # Sort nodes by their start position
            nodes_info = nodes_info.sort_values(by='start')
            # Write nodes' chrom, start, and end into one line
            line = "\t".join(nodes_info[['chrom', 'start', 'end']].astype(str).values.flatten())
            f.write(line + "\n")